<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>GeoLocation</p>
<hr style='color:#BA704F;'>

In [1]:
import geopandas as Gpd
import geodatasets as Gds
import matplotlib.pyplot as Mpl
import contextily as cx
import folium

In [56]:
#location of your file
File_Path = 'C:\\GeoLocation\\map.geojson'
File_Path1 = 'C:\\GeoLocation\\points.csv'

In [57]:
# dataframe
df1 = Gpd.read_file(File_Path1)

In [58]:
m = folium.Map(location=[32.246844749437734,-8.52468086856831], zoom_start=17)

for index,x in df1.iterrows():
    folium.Marker(
        location=[x.lat,x.lon],
        popup=x.Name,
        icon=folium.Icon(icon="star"),
    ).add_to(m)

m

In [59]:
df2 = Gpd.read_file(File_Path)

In [60]:
m = folium.Map(location=[32.246844749437734, -8.52468086856831], zoom_start=16, tiles="CartoDB positron")
for _, r in df2.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = Gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "blue"})
    folium.Popup(r["geometry"]).add_to(geo_j)
    geo_j.add_to(m)
m